In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class VAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(VAE, self).__init__()

        # Encoder
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc21 = nn.Linear(hidden_size, latent_size)
        self.fc22 = nn.Linear(hidden_size, latent_size)

        # Decoder
        self.fc3 = nn.Linear(latent_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, input_size)

    def encode(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc21(h)
        logvar = self.fc22(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc3(z))
        recon_x = torch.sigmoid(self.fc4(h))
        return recon_x

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z)
        return recon_x, mu, logvar

def loss_function(recon_x, x, mu, logvar, robust_factor=1.0, ood_factor=0.1, ood_threshold=0.1):
    # Reconstruction loss
    recon_loss = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # KL divergence
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # Robust loss (Huber loss)
    robust_loss = F.smooth_l1_loss(recon_x, x.view(-1, 784), reduction='sum')

    # OOD penalty for samples with high reconstruction error
    ood_penalty = F.relu(recon_loss - ood_threshold).sum()

    # Combine losses
    total_loss = recon_loss + kl_divergence + robust_factor * robust_loss + ood_factor * ood_penalty

    return total_loss

In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Set up a transform to normalize data between 0 and 1
transform = transforms.Compose([transforms.ToTensor()])

# Download MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

63.8%

100.0%


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100.0%
83.5%


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100.0%
100.0%


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [9]:
# Instantiate the model
input_size = 784
hidden_size = 400
latent_size = 20
vae = VAE(input_size, hidden_size, latent_size)

# Set up optimizer
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

# Training loop (assuming you have a dataset DataLoader named `train_loader`)
num_epochs = 10
for epoch in range(num_epochs):
    for batch_idx, data in enumerate(train_loader):
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data[0])
        loss = loss_function(recon_batch, data[0], mu, logvar, robust_factor=1.0, ood_factor=0.1, ood_threshold=0.1)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print('Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

Epoch 0 [0/60000 (0%)]	Loss: 44536.031250
Epoch 0 [200/60000 (11%)]	Loss: 13785.372070
Epoch 0 [400/60000 (21%)]	Loss: 11631.274414
Epoch 0 [600/60000 (32%)]	Loss: 11118.272461
Epoch 0 [800/60000 (43%)]	Loss: 9999.485352
Epoch 0 [1000/60000 (53%)]	Loss: 9769.815430
Epoch 0 [1200/60000 (64%)]	Loss: 9184.539062
Epoch 0 [1400/60000 (75%)]	Loss: 9361.836914
Epoch 0 [1600/60000 (85%)]	Loss: 8824.853516
Epoch 0 [1800/60000 (96%)]	Loss: 8268.314453
Epoch 1 [0/60000 (0%)]	Loss: 8242.572266
Epoch 1 [200/60000 (11%)]	Loss: 8423.578125
Epoch 1 [400/60000 (21%)]	Loss: 8869.374023
Epoch 1 [600/60000 (32%)]	Loss: 8283.318359
Epoch 1 [800/60000 (43%)]	Loss: 8699.750977
Epoch 1 [1000/60000 (53%)]	Loss: 8206.894531
Epoch 1 [1200/60000 (64%)]	Loss: 8124.464844
Epoch 1 [1400/60000 (75%)]	Loss: 8371.058594
Epoch 1 [1600/60000 (85%)]	Loss: 8530.292969
Epoch 1 [1800/60000 (96%)]	Loss: 8761.435547
Epoch 2 [0/60000 (0%)]	Loss: 8210.826172
Epoch 2 [200/60000 (11%)]	Loss: 8032.116699
Epoch 2 [400/60000 (21%)]	L